# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = pd.read_csv("city_data.csv")
data_file.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Köping,20,SE,1592829385,49,59.51,15.99,73.99,4.70
1,1,Solnechnyy,100,RU,1592829385,94,50.72,136.63,45.90,4.63
2,2,Mataura,0,NZ,1592829385,83,-46.19,168.86,39.00,3.00
3,3,Swabi,0,PK,1592829104,14,34.12,72.47,107.83,3.31
4,4,Rikitea,20,PF,1592829386,58,-23.12,-134.97,69.48,16.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
location = data_file[["Lat","Lng"]]
humidity = data_file["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
markers = gmaps.heatmap_layer(location)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# A max temperature lower than 80 degrees but higher than 70.
filtered_data = data_file.loc[(data_file["Max Temp"]>70) & (data_file["Max Temp"]<80)]

# Wind speed less than 10 mph
filtered_data = filtered_data.loc[(filtered_data["Wind Speed"]<10)]

# Zero cloudiness
filtered_data = filtered_data.loc[(filtered_data["Cloudiness"]==0)]

# Drop any null values and display the data
filtered_data.dropna()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
55,55,Baijiantan,0,CN,1592829392,16,45.63,85.18,79.92,8.01
167,167,Krutikha,0,RU,1592829407,60,53.96,81.21,70.23,5.37
169,169,Alta Floresta,0,BR,1592829408,73,-9.88,-56.09,77.00,1.12
203,203,Caravelas,0,BR,1592829412,57,-17.71,-39.25,77.86,4.61
218,218,Arraial do Cabo,0,BR,1592829414,78,-22.97,-42.03,75.20,3.02
240,240,Vila Velha,0,BR,1592829416,78,-20.33,-40.29,75.99,5.82
246,246,São Miguel do Araguaia,0,BR,1592829417,44,-13.28,-50.16,77.59,8.39
376,376,Arrondissement de Saverne,0,FR,1592829433,44,48.83,7.25,77.00,6.93
414,414,Senanga,0,ZM,1592829437,19,-16.12,23.27,77.95,7.27
436,436,Muyezerskiy,0,RU,1592829440,33,63.93,31.65,74.71,8.86


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = filtered_data.copy()

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
55,55,Baijiantan,0,CN,1592829392,16,45.63,85.18,79.92,8.01,
167,167,Krutikha,0,RU,1592829407,60,53.96,81.21,70.23,5.37,
169,169,Alta Floresta,0,BR,1592829408,73,-9.88,-56.09,77.00,1.12,
203,203,Caravelas,0,BR,1592829412,57,-17.71,-39.25,77.86,4.61,
218,218,Arraial do Cabo,0,BR,1592829414,78,-22.97,-42.03,75.20,3.02,


In [9]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [10]:
for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add location to params
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params = params).json()
    #print(json.dumps(response,indent = 4, sort_keys =True))
    
    try:
        hotel_df[index, "Hotel Name"] = response["results"][0]["name"] 
    except:
        print("Missing field, skipping...")

hotel_df

Missing field, skipping...
Missing field, skipping...
Missing field, skipping...


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,...,"(468, Hotel Name)","(491, Hotel Name)","(497, Hotel Name)","(523, Hotel Name)","(534, Hotel Name)","(547, Hotel Name)","(555, Hotel Name)","(560, Hotel Name)","(581, Hotel Name)","(585, Hotel Name)"
55,55,Baijiantan,0,CN,1592829392,16,45.63,85.18,79.92,8.01,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
167,167,Krutikha,0,RU,1592829407,60,53.96,81.21,70.23,5.37,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
169,169,Alta Floresta,0,BR,1592829408,73,-9.88,-56.09,77.00,1.12,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
203,203,Caravelas,0,BR,1592829412,57,-17.71,-39.25,77.86,4.61,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
218,218,Arraial do Cabo,0,BR,1592829414,78,-22.97,-42.03,75.20,3.02,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
240,240,Vila Velha,0,BR,1592829416,78,-20.33,-40.29,75.99,5.82,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
246,246,São Miguel do Araguaia,0,BR,1592829417,44,-13.28,-50.16,77.59,8.39,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
276,276,Lüderitz,0,NaN,1592829421,31,-26.65,15.16,77.00,9.17,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
376,376,Arrondissement de Saverne,0,FR,1592829433,44,48.83,7.25,77.00,6.93,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa
414,414,Senanga,0,ZM,1592829437,19,-16.12,23.27,77.95,7.27,...,Zyryanka,BON Hotel Waterfront Richards Bay,Pereybere Hotel & Spa,Hotel Imperial Edeia,Oyster Box Guesthouse,Entabeni Hotel,Goto Guesthouse Utojuku & SA Car Rental,Hotel Estância,Pousada Porto De Canoas,Veranda Paul et Virginie Hotel & Spa


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))